In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import xml.etree.ElementTree as ET
import json
import requests
import urllib
import urllib.request
import datetime
import folium
import warnings
import lightgbm
import xgboost
from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

In [2]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

code_dict={
    '전라남도 해남군': '536824B002',
    '제주도 제주시': '063057B009',
    '전라남도 무안군': '534833E001',
    '전남 해남군': '536824B002',
    '경상남도 창녕군': '635821A001',
    '전라남도 진도군': '539823A001',
    '경기도 포천시': '487915A001',
    '경상남도 밀양시': '627911A001',
    '경기도 시흥시': '429843A001',
    '강원도 철원군': '269811A001',
    '충청남도 천안시': '330846A001',
    '경상북도 청도군': '714902A001',
    '부산 강서구': '618803A001',
    '전북 남원시': '590823A001',
    '경상남도 진주시': '660985B001',
    '경상북도 상주시': '742290A001',
    '경상북도 김천시': '037268B004'
}

joosan_dict={
    '배추': '전라남도 해남군',
    '무': '제주도 제주시',
    '양파': '전라남도 무안군',
    '건고추': '전남 해남군',
    '마늘': '경상남도 창녕군',
    '대파': '전라남도 진도군',
    '얼갈이배추': '경기도 포천시',
    '양배추': '제주도 제주시',
    '깻잎': '경상남도 밀양시',
    '시금치': '경기도 포천시',
    '미나리': '경기도 시흥시',
    '당근': '제주도 제주시',
    '파프리카': '강원도 철원군',
    '새송이': '충청남도 천안시',
    '팽이버섯': '경상북도 청도군',
    '토마토': '부산 강서구',
    '청상추': '전북 남원시',
    '백다다기': '충청남도 천안시',
    '애호박': '경상남도 진주시',
    '캠벨얼리': '경상북도 상주시',
    '샤인마스캇': '경상북도 김천시'
    }

In [3]:
weather=pd.read_csv('weather.csv')

In [4]:
def preprocessing(temp_df,train, pum, len_lag, weather, joosan_dict, code_dict) :
    weather_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    # 품종과 주산지 날씨 mapping
    
    joosanji = joosan_dict[pum]
    joosan_code = code_dict[joosanji]
    joosan_weather = weather[weather['stn_Code']==joosan_code].reset_index(drop=True)
    
    end_index = np.where(joosan_weather['date']=='2020-09-28')[0][0]
    joosan_weather = joosan_weather.iloc[:end_index+1] #2020-09-28 까지만 자르기
    weather_df = weather_df.merge(joosan_weather, on='date', how='left')
    
    # 해당하는 열의 모든 값이 null 값이기 때문에 이열 두개는 삭제 처리  
    weather_df.drop(['condens_Time','gr_Temp'],axis=1,inplace=True)
    # 결측치가 존재함 -> 날씨의 특성상 앞뒤 날씨의 영향을 받거나 혹은 비슷한 시기의 날씨의 특성과 비슷할 것으로 예상
    # 따라서 결측치를 앞날의 데이터를 가져오는 것으로 결정 
    weather_df['sun_Time']=weather_df['sun_Time'].fillna(method='ffill')
    weather_df['soil_Wt']=weather_df['soil_Wt'].fillna(method='ffill')
    wea_col_list=list(weather_df.columns)
    del wea_col_list[0:6]
    #print(weather_df)
    
    # p_lag, q_lag 추가, 날씨 열 추가
    for lag in range(1,len_lag+1) :
      temp_df[f'p_lag_{lag}'] = -1
      temp_df[f'q_lag_{lag}'] = -1
      for col in range(len(wea_col_list)):
        col=wea_col_list[col]
         #or col =="widdir" or col == "wind"
        if(col == "sun_Time" or col == "sun_Qy" or col =="soil_Temp" or col =="soil_Wt" or col == "wind" or "widdir"):
            continue
        if(pum == '건고추' or pum == '당근'):
            continue
        temp_df[f'{col}_lag_{lag}'] = -1
      for index in range(lag, len(temp_df)) :
        temp_df.loc[index, f'p_lag_{lag}'] = temp_df[f'{pum}_가격(원/kg)'][index-lag] #1일전, 2일전, ... 가격을 feature로 추가
        temp_df.loc[index, f'q_lag_{lag}'] = temp_df[f'{pum}_거래량(kg)'][index-lag] #1일전, 2일전, ... 거래량을 feature로 추가
        if(pum == '건고추' or pum == '당근'):
            continue
        temp_df.loc[index, f'temp_lag_{lag}'] = weather_df['temp'][index-lag]
        temp_df.loc[index, f'max_Temp_lag_{lag}'] = weather_df['max_Temp'][index-lag]
        temp_df.loc[index, f'min_Temp_lag_{lag}'] = weather_df['min_Temp'][index-lag]
        temp_df.loc[index, f'hum_lag_{lag}'] = weather_df['hum'][index-lag]
        #temp_df.loc[index, f'widdir_lag_{lag}'] = weather_df['widdir'][index-lag]
        #temp_df.loc[index, f'wind_lag_{lag}'] = weather_df['wind'][index-lag]
        temp_df.loc[index, f'rain_lag_{lag}'] = weather_df['rain'][index-lag]
        #temp_df.loc[index, f'sun_Time_{lag}'] = weather_df['sun_Time'][index-lag]
        #temp_df.loc[index, f'sun_Qy_{lag}'] = weather_df['sun_Qy'][index-lag]
        #temp_df.loc[index, f'soil_Temp_{lag}'] = weather_df['soil_Temp'][index-lag]
        #temp_df.loc[index, f'soil_Wt_{lag}'] = weather_df['soil_Wt'][index-lag]
   
    # month 추가
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df['week']=temp_df['date'].dt.week
    temp_df['day'] = temp_df['date'].dt.weekday
    #temp_df['year'] = temp_df['date'].dt.year
    
    

    # 예측 대상(1w,2w,4w) 추가
    for week in ['1_week','2_week','4_week'] :
      temp_df[week] = 0
      n_week = int(week[0])
      for index in range(len(temp_df)) :
        try : temp_df[week][index] = temp_df[f'{pum}_가격(원/kg)'][index+7*n_week]
        except : continue

    # 불필요한 column 제거        
    temp_df = temp_df.drop(['date',f'{pum}_거래량(kg)',f'{pum}_가격(원/kg)'], axis=1)
    
    return temp_df

In [5]:
train = pd.read_csv('./train_201512~.csv')
train = train.drop('Unnamed: 0', axis = 1)
train #2015.12 ~ 2020.09

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2015-12-01,1288392.0,311.647794,855004.0,397.379895,595884.0,1349.292060,0.0,0.0,23780.0,...,7984.0,6411.0,53780.0,3371.0,72880.0,3677.0,14155.0,2090.0,0.0,0.0
1,2015-12-02,1005712.0,362.469052,687087.0,377.834757,610021.0,1376.753341,0.0,0.0,16160.0,...,7736.0,6777.0,34635.0,3554.0,61924.0,3890.0,14955.0,2091.0,0.0,0.0
2,2015-12-03,1448544.0,363.709311,837533.0,371.056543,757880.0,1287.379532,0.0,0.0,92560.0,...,8040.0,7258.0,64455.0,3574.0,80478.0,3778.0,15969.0,2080.0,0.0,0.0
3,2015-12-04,1065822.0,418.331494,582460.0,347.406088,673494.0,1259.264077,0.0,0.0,15300.0,...,8568.0,8139.0,62537.0,3722.0,85295.0,3824.0,13179.0,2168.0,0.0,0.0
4,2015-12-05,1047453.0,346.322484,617740.0,328.650565,540224.0,1175.858348,0.0,0.0,17974.0,...,9168.0,7946.0,83336.0,3471.0,100114.0,3413.0,12330.0,1928.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755,2020-09-24,1856965.0,1839.000000,2055640.0,990.000000,2281429.2,990.000000,2818.4,19101.0,134359.9,...,50730.0,4509.0,282212.3,3001.0,313139.7,3426.0,504242.6,3620.0,283196.9,10940.0
1756,2020-09-25,1880095.5,1789.000000,1879261.0,1011.000000,2074513.0,955.000000,1887.1,23095.0,126926.0,...,54322.0,4178.0,312214.8,2999.0,362741.0,3357.0,479683.1,3618.0,303779.6,10844.0
1757,2020-09-26,1661090.9,1760.000000,1709385.7,1075.000000,2089081.2,961.000000,959.0,22510.0,110357.7,...,61213.0,3770.0,327395.8,3065.0,390361.2,3092.0,521493.8,3691.0,313295.7,10636.0
1758,2020-09-27,25396.0,3066.000000,38222.0,1139.000000,18240.0,1056.000000,60.0,22333.0,620.0,...,144.0,4076.0,285.0,3707.0,2464.0,3252.0,21717.0,3567.0,9734.0,10699.0


In [7]:
def model_train(x_train, y_train, x_valid, y_valid) :
    params = {'learning_rate': 0.01, 
              'max_depth': 6, 
              'booster': 'gbtree', 
              'objective': 'reg:squarederror',  
              'max_leaves': 100, 
              'colsample_bytree': 0.8, 
              'subsample': 0.8,  
              'seed':42,
             }
   # train_data = xgb.DMatrix(train_features, label=train_y)
    valid_data = xgboost.DMatrix(x_valid, label=y_valid)
    
    model = xgboost.train(params, 
                   dtrain = xgboost.DMatrix(data = x_train, label = y_train),
                   num_boost_round = 10000,
                   evals = [(valid_data, 'eval')],
                   early_stopping_rounds = 80,
                    )
    return model

In [ ]:
model_dict = {}
split = 28 #validation

for pum in tqdm(unique_pum + unique_kind):
    # 품목 품종별 전처리
    temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    temp_df = preprocessing(temp_df,train, pum, 28, weather, joosan_dict, code_dict)
    #temp_df['year'] = temp_df['year'].map(stat_mapping)
    
    #temp_df = temp_df.drop('month', axis = 1)
    #temp_df = temp_df.drop('week', axis = 1)
    
    
    # 주차별(1,2,4w) 학습
    for week_num in [1,2,4] :
        x = temp_df[temp_df[f'{week_num}_week']>0].iloc[28:,:-3]
        y = temp_df[temp_df[f'{week_num}_week']>0].iloc[28:][f'{week_num}_week']
        
        #train, test split
        # shuffle = False

        #x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.1, shuffle=True, random_state=42)
        x_train = x[:-split]
        y_train = y[:-split]
        x_valid = x[-split:]
        y_valid = y[-split:]
        
        model_dict[f'{pum}_model_{week_num}'] = model_train(x_train, y_train, x_valid, y_valid)
        #print(pum,"성능", model_dict[f'{pum}_model_{week_num}'].score(x_valid,y_valid))

  0%|                                                                                           | 0/21 [00:00<?, ?it/s]

[0]	eval-rmse:1794.06287
[1]	eval-rmse:1771.64331
[2]	eval-rmse:1750.00476
[3]	eval-rmse:1727.62671
[4]	eval-rmse:1707.37463
[5]	eval-rmse:1689.06543
[6]	eval-rmse:1669.27637
[7]	eval-rmse:1650.74841
[8]	eval-rmse:1632.03564
[9]	eval-rmse:1616.22729
[10]	eval-rmse:1598.35290
[11]	eval-rmse:1579.44690
[12]	eval-rmse:1562.68994
[13]	eval-rmse:1547.49255
[14]	eval-rmse:1532.00757
[15]	eval-rmse:1515.45215
[16]	eval-rmse:1499.00147
[17]	eval-rmse:1485.97864
[18]	eval-rmse:1469.05627
[19]	eval-rmse:1453.50806
[20]	eval-rmse:1440.97742
[21]	eval-rmse:1427.84583
[22]	eval-rmse:1415.47058
[23]	eval-rmse:1402.18469
[24]	eval-rmse:1388.61487
[25]	eval-rmse:1374.11279
[26]	eval-rmse:1362.44910
[27]	eval-rmse:1348.87158
[28]	eval-rmse:1337.10742
[29]	eval-rmse:1324.60254
[30]	eval-rmse:1313.98315
[31]	eval-rmse:1304.21533
[32]	eval-rmse:1291.02576
[33]	eval-rmse:1280.70374
[34]	eval-rmse:1268.56958
[35]	eval-rmse:1257.97815
[36]	eval-rmse:1247.63989
[37]	eval-rmse:1237.35413
[38]	eval-rmse:1227.14

[317]	eval-rmse:765.44641
[318]	eval-rmse:765.61444
[319]	eval-rmse:765.70123
[320]	eval-rmse:765.66113
[321]	eval-rmse:765.62561
[322]	eval-rmse:765.52667
[323]	eval-rmse:765.65839
[324]	eval-rmse:765.80102
[325]	eval-rmse:765.92499
[326]	eval-rmse:765.78162
[327]	eval-rmse:765.52643
[328]	eval-rmse:765.52631
[329]	eval-rmse:765.53601
[330]	eval-rmse:765.64166
[331]	eval-rmse:765.77972
[332]	eval-rmse:765.85535
[333]	eval-rmse:766.12054
[334]	eval-rmse:765.99475
[335]	eval-rmse:765.98633
[336]	eval-rmse:765.84387
[337]	eval-rmse:765.82214
[338]	eval-rmse:765.83807
[339]	eval-rmse:765.68945
[340]	eval-rmse:765.79089
[341]	eval-rmse:765.64087
[342]	eval-rmse:765.52075
[343]	eval-rmse:765.53424
[344]	eval-rmse:765.42114
[345]	eval-rmse:765.38763
[346]	eval-rmse:765.23962
[347]	eval-rmse:765.35113
[348]	eval-rmse:765.45251
[349]	eval-rmse:765.54828
[350]	eval-rmse:765.53375
[351]	eval-rmse:765.54010
[352]	eval-rmse:765.57159
[353]	eval-rmse:765.24432
[354]	eval-rmse:765.08612
[355]	eval-r

[197]	eval-rmse:853.85602
[198]	eval-rmse:854.14117
[199]	eval-rmse:853.63293
[200]	eval-rmse:853.31714
[201]	eval-rmse:853.21692
[202]	eval-rmse:852.45068
[203]	eval-rmse:852.08063
[204]	eval-rmse:852.17548
[205]	eval-rmse:851.48962
[206]	eval-rmse:850.78540
[207]	eval-rmse:850.44898
[208]	eval-rmse:850.03210
[209]	eval-rmse:849.67645
[210]	eval-rmse:849.45947
[211]	eval-rmse:848.99969
[212]	eval-rmse:848.43079
[213]	eval-rmse:848.15387
[214]	eval-rmse:847.92242
[215]	eval-rmse:847.54553
[216]	eval-rmse:847.31079
[217]	eval-rmse:847.22363
[218]	eval-rmse:847.14154
[219]	eval-rmse:847.04669
[220]	eval-rmse:846.81097
[221]	eval-rmse:846.59357
[222]	eval-rmse:846.19354
[223]	eval-rmse:845.97186
[224]	eval-rmse:845.61511
[225]	eval-rmse:845.37726
[226]	eval-rmse:845.11719
[227]	eval-rmse:844.76117
[228]	eval-rmse:844.81683
[229]	eval-rmse:844.67828
[230]	eval-rmse:844.60052
[231]	eval-rmse:844.33032
[232]	eval-rmse:843.78845
[233]	eval-rmse:843.58984
[234]	eval-rmse:843.48956
[235]	eval-r

[381]	eval-rmse:911.50531
[382]	eval-rmse:911.59375
[383]	eval-rmse:911.51434
[384]	eval-rmse:911.45306
[385]	eval-rmse:911.46289
[386]	eval-rmse:911.21344
[387]	eval-rmse:911.11609
[388]	eval-rmse:911.11163
[389]	eval-rmse:911.03381
[390]	eval-rmse:910.99780
[391]	eval-rmse:911.02978
[392]	eval-rmse:911.03925
[393]	eval-rmse:911.04266
[394]	eval-rmse:911.01904
[395]	eval-rmse:910.99554
[396]	eval-rmse:910.90826
[397]	eval-rmse:910.63110
[398]	eval-rmse:910.55609
[399]	eval-rmse:910.48669
[400]	eval-rmse:910.60059
[401]	eval-rmse:910.66113
[402]	eval-rmse:910.65570
[403]	eval-rmse:910.58215
[404]	eval-rmse:910.40314
[405]	eval-rmse:910.41565
[406]	eval-rmse:910.29187
[407]	eval-rmse:910.21668
[408]	eval-rmse:910.18127
[409]	eval-rmse:910.06836
[410]	eval-rmse:910.00452
[411]	eval-rmse:909.97168
[412]	eval-rmse:910.03430
[413]	eval-rmse:909.99426
[414]	eval-rmse:909.93292
[415]	eval-rmse:909.92181
[416]	eval-rmse:909.83453
[417]	eval-rmse:909.73126
[418]	eval-rmse:909.64770
[419]	eval-r

[697]	eval-rmse:904.19275
[698]	eval-rmse:904.22308
[699]	eval-rmse:904.23944
[700]	eval-rmse:904.18964
[701]	eval-rmse:904.21094
[702]	eval-rmse:904.22650
[703]	eval-rmse:904.20648
[704]	eval-rmse:904.20923
[705]	eval-rmse:904.20319
[706]	eval-rmse:904.19305
[707]	eval-rmse:904.16315
[708]	eval-rmse:904.06250
[709]	eval-rmse:904.09790
[710]	eval-rmse:904.02893
[711]	eval-rmse:904.06079
[712]	eval-rmse:904.07105
[713]	eval-rmse:904.10278
[714]	eval-rmse:904.10120
[715]	eval-rmse:904.12695
[716]	eval-rmse:904.15241
[717]	eval-rmse:904.13599
[718]	eval-rmse:904.18939
[719]	eval-rmse:904.20038
[720]	eval-rmse:904.11926
[721]	eval-rmse:904.14691
[722]	eval-rmse:904.19501
[723]	eval-rmse:904.19739
[724]	eval-rmse:904.17895
[725]	eval-rmse:904.20081
[726]	eval-rmse:904.22217
[727]	eval-rmse:904.21985
[728]	eval-rmse:904.19885
[729]	eval-rmse:904.16815
[730]	eval-rmse:904.15869
[731]	eval-rmse:904.15979
[732]	eval-rmse:904.12891
[733]	eval-rmse:904.17480
[734]	eval-rmse:904.15411
[735]	eval-r